In [1]:
using Statistics
# using CuArrays
using Flux, Flux.Tracker, Flux.Optimise
using Metalhead, Images
using Metalhead: trainimgs
using Images.ImageCore
using Flux: onehotbatch, onecold
using Base.Iterators: partition

┌ Info: Recompiling stale cache file /root/.julia/compiled/v1.2/Flux/QdkVy.ji for Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1240
┌ Info: Recompiling stale cache file /root/.julia/compiled/v1.2/Metalhead/OYscp.ji for Metalhead [dbeba491-748d-5e0e-a39e-b530a07fa0cc]
└ @ Base loading.jl:1240


In [5]:
# 先下载 CIFAR10数据集，并且将其分batch
Metalhead.download(CIFAR10)
X = trainimgs(CIFAR10)
labels = onehotbatch([X[i].ground_truth.class for i in 1:50000],1:10)

# 随机显示其中的图片
image(x) = x.img
ground_truth(x) = x.ground_truth
# image.(X[rand(1:end, 10)])

ground_truth (generic function with 1 method)

In [42]:
?gpu

search: gpu thin_edges_nonmaxsup_subpix gaussian_pyramid



No documentation found.

`Flux.gpu` is a `Function`.

```
# 1 method for generic function "gpu":
[1] gpu(x) in Flux at /root/.julia/packages/Flux/dkJUV/src/treelike.jl:69
```


In [8]:
getarray(X) = float.(permutedims(channelview(X), (2, 3, 1)))
imgs = [getarray(X[i].img) for i in 1:50000]

train = gpu.([(cat(imgs[i]..., dims = 4), labels[:,i]) for i in partition(1:49000, 1000)])
valset = 49001:50000
valX = cat(imgs[valset]..., dims = 4) |> gpu
valY = labels[:, valset] |> gpu

10×1000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  0  0  1  0  1  0  0  0  0  0  0  …  0  0  0  0  1  0  1  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  1  0  0  0  1  0  0  0  0  1  1
 0  0  0  0  0  0  0  0  1  0  0  0  0     0  0  0  1  0  0  0  1  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  0  0     0  0  1  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0  …  1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  1  0  0  0  1  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  1  0  1  0  0  0  0  0  0  0  0  1     0  0  0  0  0  0  0  0  0  1  0  0

In [43]:
m = Chain(
  Conv((5,5), 3=>16, relu),
  MaxPool((2,2)),
  Conv((5,5), 16=>8, relu),
  MaxPool((2,2)),
  x -> reshape(x, :, size(x, 4)),
  Dense(200, 120),
  Dense(120, 84),
  Dense(84, 10),
  softmax) |> gpu

Chain(Conv((5, 5), 3=>16, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((5, 5), 16=>8, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), #17, Dense(200, 120), Dense(120, 84), Dense(84, 10), softmax)

In [44]:
using Flux: crossentropy, Momentum

loss(x, y) = sum(crossentropy(m(x), y))
opt = Momentum(params(m), 0.01)

┌ Warning: Momentum(params) is deprecated; use Momentum(η::Float64) instead
│   caller = top-level scope at In[44]:4
└ @ Core In[44]:4


#24 (generic function with 1 method)

In [45]:
accuracy(x, y) = mean(onecold(m(x), 1:10) .== onecold(y, 1:10))

accuracy (generic function with 1 method)

In [48]:
epochs = 50

for epoch = 1:epochs
  for d in train
    l = loss(d...)    # d展开之后就是对应的两个参数
    back!(l)
    opt()
  end
  @show accuracy(valX, valY)
end

accuracy(valX, valY) = 0.324
accuracy(valX, valY) = 0.374
accuracy(valX, valY) = 0.389
accuracy(valX, valY) = 0.404
accuracy(valX, valY) = 0.429
accuracy(valX, valY) = 0.432
accuracy(valX, valY) = 0.437
accuracy(valX, valY) = 0.453
accuracy(valX, valY) = 0.46
accuracy(valX, valY) = 0.443
accuracy(valX, valY) = 0.447
accuracy(valX, valY) = 0.462
accuracy(valX, valY) = 0.461
accuracy(valX, valY) = 0.475
accuracy(valX, valY) = 0.478
accuracy(valX, valY) = 0.49
accuracy(valX, valY) = 0.493
accuracy(valX, valY) = 0.501
accuracy(valX, valY) = 0.506
accuracy(valX, valY) = 0.507
accuracy(valX, valY) = 0.512
accuracy(valX, valY) = 0.507
accuracy(valX, valY) = 0.513
accuracy(valX, valY) = 0.514
accuracy(valX, valY) = 0.515
accuracy(valX, valY) = 0.514
accuracy(valX, valY) = 0.508
accuracy(valX, valY) = 0.517
accuracy(valX, valY) = 0.52
accuracy(valX, valY) = 0.521
accuracy(valX, valY) = 0.519
accuracy(valX, valY) = 0.518
accuracy(valX, valY) = 0.526
accuracy(valX, valY) = 0.527
accuracy(valX, va